In [3]:
from datetime import datetime
startTime = datetime.now()

from collections import defaultdict
import sys

filepath = 'input.txt'

INSTRUCTION_JUMPS = {'01': 4, '02': 4, '03': 2, '04': 2, '05': 3, '06': 3, '07': 4, '08': 4, '09': 2}
MODES = {'0': lambda i, n, data, relative_base: data[data[i+n]], 
         '1': lambda i, n, data, relative_base: data[i+n],
         '2': lambda i, n, data, relative_base: data[data[i+n] + relative_base]}

inputs = []
def run_program(data):
    global inputs
    data = data + [0] * 1000
    i, num_inputs = 0, 0
    relative_base = 0
    
    while data[i] != 99:
        
        opcode = str(data[i])[-2:].zfill(2)
        mode = str(data[i])[:-2].zfill(3)
        #print(opcode)
        #print(i)
        #print(data[i])
        
        
        if opcode in ('01', '02', '04', '05', '06', '07', '08', '09'):
            p1 = MODES[mode[2]](i, 1, data, relative_base)     
        if opcode in ('01', '02', '05', '06', '07', '08'):
            p2 = MODES[mode[1]](i, 2, data, relative_base)
            
        
        
        if opcode in ['03']:
            if mode[2] == '2':
                p3 = data[i+1] + relative_base
            else:
                p3 = data[i+1]
            data = data + [0] * (p3+1-len(data))                  
            
        if opcode in ['01', '02', '07', '08']:
            if mode[0] == '2':
                p3 = data[i+3] + relative_base
            else:
                p3 = data[i+3]
            data = data + [0] * (p3+1-len(data))

        if opcode == '01':
            data[p3] = p1 + p2
        elif opcode == '02':
            data[p3] = p1 * p2
        elif opcode == '03':
            #print('taking input')
            
            #for line in grid[:30]:
            #    print(''.join(['#' if i == 1 else '_' if i == 3 else 'O' if i == 4 else 'X' if i == 2 else ' ' for i in line]))
            #print(inputs[-1])
            data[p3] = inputs[-1]
            inputs = []
            
            
            #print(data[p3]) 
            #data[p3] = inputs[num_inputs]
            #print(inputs[num_inputs])
            #num_inputs += 1
        elif opcode == '04':
            #print('yielding', p1)
            yield p1 
        elif opcode == '05' and p1 != 0:
            i = p2
            continue
        elif opcode == '06' and p1 == 0:
            i = p2
            continue 
        elif opcode == '07':
            data[p3] = int(p1 < p2) 
        elif opcode == '08':
            data[p3] = int(p1 == p2)
        elif opcode == '09':
            relative_base += p1
        else:
            pass
            
        i += INSTRUCTION_JUMPS[opcode]
        
    yield None
    
    
ADJACENTS = {1:(0, 1), 2: (0, -1), 3: (-1, 0), 4: (1, 0)}
visited = {}
visited[0, 0] = sys.maxsize

def run():
    global inputs
    
    opposite = {1:2, 2:1, 3:4, 4:3}
    
    with open(filepath, 'r') as f:
        for line in f.readlines():
            data = [int(i) for i in line.strip().split(',')]
    f.close()

    program = run_program(data[:])
    unvisited = [] # distance from origin, movement path from origin
    unvisited.append([0, [], 0, 0])
    

    try:
        while unvisited:
            current = unvisited[0]
            for position in unvisited[1:]:
                if position[0] < position[0]:
                    current = position

            #print(current)
            
            program = run_program(data[:])
            inputs = []

            return_to_pos = current[1][:]
            while return_to_pos:
                inputs.append(return_to_pos[0])
                next(program)
                del return_to_pos[0]
                
            for direction in range(1, 5):
                if current[0] != 0 and direction == opposite[current[1][-1]]:
                    continue
                inputs.append(direction)
                out = next(program)
                #print(direction, out)
                
                if out == 0:
                    pass
                elif out == 1:
                    x = current[2] + ADJACENTS[direction][0]
                    y = current[3] + ADJACENTS[direction][1]
                    
                    unvisited.append([current[0] + 1, current[1] + [direction], x, y])
                    visited[x, y] = sys.maxsize
                    inputs.append(opposite[direction])
                    next(program)
                    
                elif out == 2:
                    x = current[2] + ADJACENTS[direction][0]
                    y = current[3] + ADJACENTS[direction][1]
                    
                    unvisited.append([current[0] + 1, current[1] + [direction], x, y])
                    visited[x, y] = 0
                    inputs.append(opposite[direction])
                    
                    next(program)
                    print('OUTPUTTING', current[0] + 1)
                    #return True
                else:
                    print('sad monkey')

            unvisited.remove(current)
            #print(unvisited)
            #print()
    except StopIteration:
        print('exiting')
        pass

run()
print('done p1')

# BFS until a bot hits oxygen supply
# reset path length and set of already explored tiles
# use same bot still parked at oxygen to BFS until ran out of spaces to explore

OUTPUTTING 204
done p1


In [10]:
a = visited.copy()
for key, val in a.items():
    if val == 0:
        start = key
        
max_distance = 0
print(start)

ADJACENTS = {1:(0, 1), 2: (0, -1), 3: (-1, 0), 4: (1, 0)}
        
while a:
    cur_pos = (list(a.keys()))[0]
    cur_dis = a[cur_pos]
    for key in a.keys():
        if a[key] < cur_dis:
            cur_pos = key
            cur_dis = a[key]
            
    #print(cur_pos, cur_dis)
    
    if cur_dis > max_distance:
        max_distance = cur_dis

    for adjacent in ADJACENTS.values():
        x = cur_pos[0] + adjacent[0]
        y = cur_pos[1] + adjacent[1]

        for key in a.keys():
            if key == (x, y):
                a[key] = cur_dis + 1
                #print(key)
                break
    del a[cur_pos]
    
print(max_distance)

(-16, -12)
340


In [2]:
import numpy, sys, copy
from datetime import datetime
startTime = datetime.now()
grid = numpy.zeros((0, 0))

ADJACENTS = {(0, -1), (0, 1), (-1, 0), (1, 0)}

class Node():
    def __init__(self, position = None, distance = sys.maxsize):
        self.position = position
        self.distance = distance
        
    def __eq__(self, other):
        return self.position == other.position
    
def pathfind(start, end, unvisited):
    t = datetime.now()
    for node in unvisited:
        if node.position == start:
            node.distance = 0
            break
            
    while unvisited:
        current_node = unvisited[0]
        for node in unvisited[1:]:
            if node.distance < current_node.distance:
                current_node = node

        if current_node.position == end:
            print(datetime.now()-t)
            return current_node.distance

        for adjacent in ADJACENTS:
            x, y = current_node.position[0] + adjacent[0], current_node.position[1] + adjacent[1]

            for node in unvisited:
                if node.position == (x, y) and node.distance > current_node.distance + 1:
                    node.distance = current_node.distance + 1
                    break

        unvisited.remove(current_node)
    
locations = {}
unvisited = []
x, y = 0, 0
with open('input.txt', 'r') as f:
    for line in f.readlines():
        for x, n in enumerate(line.strip()):
            if n != '#':
                unvisited.append(Node((y, x)))    
            if n.isdigit():
                locations[int(n)] = y, x
        y += 1
f.close()

distances = {}
for start_point in locations:
    for end_point in locations:
        if start_point != end_point and tuple(sorted([start_point, end_point])) not in distances.keys():
            new_path = copy.deepcopy(unvisited)
            distances[tuple(sorted([start_point, end_point]))] = pathfind(locations[start_point], locations[end_point], new_path)

from itertools import permutations
routes = permutations(range(1, int(max(locations.keys())) + 1))
min_distance_p1, min_distance_p2 = float('inf'), float('inf')
for route in routes:
    distance = 0
    start_point = 0
   
    for pos in route:
        distance += distances[tuple(sorted([start_point, pos]))]
        start_point = pos
   
    if distance < min_distance_p1:
        min_distance_p1 = distance
        
    distance += distances[tuple(sorted([start_point, 0]))]
    
    if distance < min_distance_p2:
        min_distance_p2 = distance
    
# number of distances = n! / 2*(n-2)!
# number of combinations stemming from 0 = (n-1)!
print(min_distance_p1, min_distance_p2)
print(str(datetime.now() - startTime)[:-3])


0:00:00.069516
0:00:00.100111
0:00:00.052125
0:00:00.050947
0:00:00.034255
0:00:00.006995
0:00:00.199457
0:00:00.013448
0:00:00.015251
0:00:00.040755
0:00:00.104059
0:00:00.028728
0:00:00.115379
0:00:00.083238
0:00:00.232472
0:00:00.058454
0:00:00.062749
0:00:00.096670
0:00:00.037413
0:00:00.081197
0:00:00.054862


KeyboardInterrupt: 

[0, []]


In [26]:
current = [1, [1234]]
print(current[1][-1])


1234


In [1]:
from datetime import datetime
startTime = datetime.now()

from collections import defaultdict

filepath = 'input.txt'

INSTRUCTION_JUMPS = {'01': 4, '02': 4, '03': 2, '04': 2, '05': 3, '06': 3, '07': 4, '08': 4, '09': 2}
MODES = {'0': lambda i, n, data, relative_base: data[data[i+n]], 
         '1': lambda i, n, data, relative_base: data[i+n],
         '2': lambda i, n, data, relative_base: data[data[i+n] + relative_base]}

inputs = []
def run_program(data):
    global inputs
    data = data + [0] * 1000
    i, num_inputs = 0, 0
    relative_base = 0
    
    while data[i] != 99:
        
        opcode = str(data[i])[-2:].zfill(2)
        mode = str(data[i])[:-2].zfill(3)
        #print(opcode)
        #print(i)
        #print(data[i])
        
        
        if opcode in ('01', '02', '04', '05', '06', '07', '08', '09'):
            p1 = MODES[mode[2]](i, 1, data, relative_base)     
        if opcode in ('01', '02', '05', '06', '07', '08'):
            p2 = MODES[mode[1]](i, 2, data, relative_base)
            
        
        
        if opcode in ['03']:
            if mode[2] == '2':
                p3 = data[i+1] + relative_base
            else:
                p3 = data[i+1]
            data = data + [0] * (p3+1-len(data))                  
            
        if opcode in ['01', '02', '07', '08']:
            if mode[0] == '2':
                p3 = data[i+3] + relative_base
            else:
                p3 = data[i+3]
            data = data + [0] * (p3+1-len(data))

        if opcode == '01':
            data[p3] = p1 + p2
        elif opcode == '02':
            data[p3] = p1 * p2
        elif opcode == '03':
            #print('taking input')
            
            #for line in grid[:30]:
            #    print(''.join(['#' if i == 1 else '_' if i == 3 else 'O' if i == 4 else 'X' if i == 2 else ' ' for i in line]))
            #print(inputs[-1])
            data[p3] = inputs[-1]
            inputs = []
            
            
            #print(data[p3]) 
            #data[p3] = inputs[num_inputs]
            #print(inputs[num_inputs])
            #num_inputs += 1
        elif opcode == '04':
            #print('yielding', p1)
            yield p1 
        elif opcode == '05' and p1 != 0:
            i = p2
            continue
        elif opcode == '06' and p1 == 0:
            i = p2
            continue 
        elif opcode == '07':
            data[p3] = int(p1 < p2) 
        elif opcode == '08':
            data[p3] = int(p1 == p2)
        elif opcode == '09':
            relative_base += p1
        else:
            pass
            
        i += INSTRUCTION_JUMPS[opcode]
        
    yield None
    
    
ADJACENTS = {(0, -1), (0, 1), (-1, 0), (1, 0)}

def run():
    global inputs
    
    opposite = {1:2, 2:1, 3:4, 4:3}
    
    with open(filepath, 'r') as f:
        for line in f.readlines():
            data = [int(i) for i in line.strip().split(',')]
    f.close()

    program = run_program(data[:])
    unvisited = [] # distance from origin, movement path from origin
    unvisited.append([0, [], 0, 0])
    visited = defaultdict(int)
    visited[0, 0] = sys.maxsize

    try:
        for i in range(1000): #while unvisited:
            current = unvisited[0]
            for position in unvisited[1:]:
                if position[0] < position[0]:
                    current = position

            print(current)
            
            program = run_program(data[:])
            inputs = []

            return_to_pos = current[1][:]
            while return_to_pos:
                inputs.append(return_to_pos[0])
                next(program)
                del return_to_pos[0]
                
            for direction in range(1, 5):
                if current[0] != 0 and direction == opposite[current[1][-1]]:
                    continue
                inputs.append(direction)
                out = next(program)
                print(direction, out)
                
                if out == 0:
                    pass
                elif out == 1:
                    x = current_node.position[0] + adjacent[0], 
                    y = current_node.position[1] + adjacent[1]
                    
                    unvisited.append([current[0] + 1, current[1] + [direction]])
                    inputs.append(opposite[direction])
                    next(program)
                    
                elif out == 2:
                    print('OUTPUTTING', current[0] + 1)
                    return True
                else:
                    print('sad monkey')

            unvisited.remove(current)
            print(unvisited)
            print()
    except StopIteration:
        print('exiting')
        pass

run()

NameError: name 'sys' is not defined

In [84]:
b = {1:4, 2:3, 3:3}
print(b.keys())

dict_keys([1, 2, 3])
